# **Tiempo mercado**

In [1]:
# Manejo de datos
import os # Directorios
import pandas as pd # Manipulación df
# Gráficas
import plotly.graph_objects as go #Para obtener librería usar: pip install plotly
from plotly.subplots import make_subplots
import plotly.io as pio # Exportar gráfica

In [2]:
# Obtener el directorio actual de trabajo
directorio_actual = os.getcwd()
# Directorio donde se encuentran los archivos JSON (ruta relativa)
directorio_json = os.path.join(directorio_actual, 'datos_json')
#print("Directorio JSON relativo:", directorio_json)
# Obtener la lista de archivos JSON en el directorio
archivos_json = os.listdir(directorio_json)

In [3]:
dataframes = {}
# Iterar sobre cada archivo JSON y crear un DataFrame
for archivo in archivos_json:
    # Obtener el nombre de la tabla del nombre del archivo
    nombre_tabla = archivo.replace('datos_', '').replace('.json', '')    
    # Cargar el archivo JSON en un DataFrame y asignarlo a una variable con un nombre dinámico
    ruta_json = os.path.join(directorio_json, archivo)
    globals()[f"df_{nombre_tabla}"] = pd.read_json(ruta_json)

In [4]:
# Obtener todos los nombres de las variables globales
nombres_variables_globales = list(globals().keys())
# Filtrar los nombres, solo aquellos que comienzan con "df_"
nombres_df = [nombre for nombre in nombres_variables_globales if nombre.startswith("df_")]

# Lista de nombres de los DataFrames creados
print("Lista de DataFrames creados:")
print(nombres_df)
dfs_list = nombres_df 

Lista de DataFrames creados:
['df_alfa_q_feb_2023_pachuca', 'df_alfa_q_jul_2022_tulancingo', 'df_alfa_q_jul_2023_pachuca', 'df_alfa_q_jul_2023_tulancingo', 'df_alfa_q_jun_2023_pachuca', 'df_alfa_q_jun_2023_tulancingo', 'df_alfa_q_mar_2023_pachuca', 'df_alfa_q_mar_2023_tulancingo', 'df_alfa_q_may_2022_tulancingo', 'df_alfa_q_may_2023_tulancingo', 'df_alfa_q_nov_2022_pachuca', 'df_alfa_q_oct_2022_pachuca', 'df_alfa_q_oct_2022_tulancingo', 'df_alfa_q_oct_2023_tulancingo', 'df_alfa_q_puebla', 'df_alfa_q_sep_2023_pachuca', 'df_alfa_q_sep_2023_tulancingo', 'df_enero_2024_querertaro', 'df_financiamientos_2019_pachuca', 'df_financiamientos_2019_puebla', 'df_financiamientos_2019_tulancingo', 'df_financiamientos_2020_pachuca', 'df_financiamientos_2020_puebla', 'df_financiamientos_2020_tulancingo', 'df_financiamientos_2021_pachuca', 'df_financiamientos_2021_puebla', 'df_financiamientos_2021_tulancingo', 'df_financiamientos_2022_pachuca', 'df_financiamientos_2022_puebla', 'df_financiamientos_2022_

# *PACHUCA*

### BASE

In [5]:
df_mercado_pachuca = df_publicacion_pachuca[['id','tiempo_de_publicacion']]
df_mercado_pachuca.head()

,id,tiempo_de_publicacion
0,1,72
1,2,119
2,3,293
3,4,293
4,5,273


In [6]:
# Definir la función de categorización
def categorizar(valor):
    if valor.isdigit():  # Verificar si el valor es un número
        valor = int(valor)  # Convertir a entero si es un número
        if 0 <= valor <= 10:
            return '0-10'
        elif 11 <= valor <= 30:
            return '11-30'
        elif 31 <= valor <= 100:
            return '31-100'
        elif 101 <= valor <= 364:
            return '101-364'
        else:
            return '>=365'
    else:
        return 'No numérico'  # Manejar valores no numéricos
# Lista de nombres de los DataFrames
dataframes_name = ['df_mercado_pachuca']

# Lista de DataFrames
dataframes_list = [df_mercado_pachuca]

# Definir el orden de las categorías
todas_categorias = ['0-10', '11-30', '31-100', '101-364', '>=365']

# Diccionario para almacenar los DataFrames de conteo por nombre de DataFrame original
dataframes_conteo_por_nombre = {}

# Iterar sobre cada DataFrame
for name, df in zip(dataframes_name, dataframes_list):
    # Aplicar la función de categorización
    df['dias'] = df['tiempo_de_publicacion'].apply(categorizar)
    # Contar el número de registros únicos por cada valor único en la columna 'dias'
    conteo_por_dias = df['dias'].value_counts().reindex(todas_categorias, fill_value=0)
    # Crear un nuevo DataFrame de conteo
    df_conteo = pd.DataFrame({'dias': conteo_por_dias.index, 'conteo': conteo_por_dias.values})
    # Ordenar el DataFrame según el orden definido en todas_categorias
    df_conteo['dias'] = pd.Categorical(df_conteo['dias'], categories=todas_categorias, ordered=True)
    df_conteo = df_conteo.sort_values(by='dias')
    # Almacenar el DataFrame de conteo en el diccionario por nombre de DataFrame original
    dataframes_conteo_por_nombre[name] = df_conteo
    # Imprimir el nombre del DataFrame de conteo y su contenido
    nuevo_nombre_df_conteo = 'conteo_' + name
    globals()[nuevo_nombre_df_conteo] = df_conteo
    print(f'Nombre del DataFrame de conteo: {nuevo_nombre_df_conteo}')
    print(df_conteo)

Nombre del DataFrame de conteo: conteo_df_mercado_pachuca
      dias  conteo
0     0-10     162
1    11-30     169
2   31-100     418
3  101-364     905
4    >=365     411


C:\Users\yoe11\AppData\Local\Temp\ipykernel_13744\3192967569.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['dias'] = df['tiempo_de_publicacion'].apply(categorizar)


In [7]:
values = conteo_df_mercado_pachuca['conteo']
labels = conteo_df_mercado_pachuca['dias']
colores = ['#2962ff', '#9500ff', '#ff0059', '#ff8c00', '#b4e600', '#2EC2A2','#E50CB6']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=labels, 
    y=values, 
    marker_color=colores,  # Especifica los colores de las barras
    text=values,  # Texto que se mostrará en las barras 
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    #title='Número de unidades por días en el mercado',  # Título de la gráfica
    xaxis=dict(title='Rango días'),  # Título del eje x
    yaxis=dict(
        title='Unidades', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_tiempoXmercado_pachuca', carpeta='graficas')
fig.show()

# *TULANCINGO*

In [10]:
df_tiempomercado_tulancingo = df_tiempo_dias_tulancingo[['id', 'tiempo_publicacion']]

In [11]:
# Definir la función de categorización
def categorizar(valor):
    valor = int(valor)  # Convertir a entero si es un número
    if 0 <= valor <= 10:
        return '0-10'
    elif 11 <= valor <= 30:
        return '11-30'
    elif 31 <= valor <= 100:
        return '31-100'
    elif 101 <= valor <= 364:
        return '101-364'
    else:
        return '>=365'
# Lista de nombres de los DataFrames
dataframes_name = ['df_tiempo_dias_tulancingo']

# Lista de DataFrames
dataframes_list = [df_tiempo_dias_tulancingo]

# Definir el orden de las categorías
todas_categorias = ['0-10', '11-30', '31-100', '101-364', '>=365']

# Diccionario para almacenar los DataFrames de conteo por nombre de DataFrame original
dataframes_conteo_por_nombre = {}

# Iterar sobre cada DataFrame
for name, df in zip(dataframes_name, dataframes_list):
    # Aplicar la función de categorización
    df['dias'] = df['tiempo_publicacion'].apply(categorizar)
    # Contar el número de registros únicos por cada valor único en la columna 'dias'
    conteo_por_dias = df['dias'].value_counts().reindex(todas_categorias, fill_value=0)
    # Crear un nuevo DataFrame de conteo
    tiempo_mercado_tulancingo = pd.DataFrame({'dias': conteo_por_dias.index, 'conteo': conteo_por_dias.values})
    # Ordenar el DataFrame según el orden definido en todas_categorias
    tiempo_mercado_tulancingo['dias'] = pd.Categorical(tiempo_mercado_tulancingo['dias'], categories=todas_categorias, ordered=True)
    tiempo_mercado_tulancingo = tiempo_mercado_tulancingo.sort_values(by='dias')
    # Almacenar el DataFrame de conteo en el diccionario por nombre de DataFrame original
    dataframes_conteo_por_nombre[name] = tiempo_mercado_tulancingo
    # Imprimir el nombre del DataFrame de conteo y su contenido
    nuevo_nombre_df_conteo = 'conteo_' + name
    globals()[nuevo_nombre_df_conteo] = tiempo_mercado_tulancingo
    print(f'Nombre del DataFrame de conteo: {nuevo_nombre_df_conteo}')
    print(tiempo_mercado_tulancingo)

Nombre del DataFrame de conteo: conteo_df_tiempo_dias_tulancingo
      dias  conteo
0     0-10       0
1    11-30      63
2   31-100      24
3  101-364       0
4    >=365       0


In [12]:
values = tiempo_mercado_tulancingo['conteo']
labels = tiempo_mercado_tulancingo['dias']
colores = ['#3d45c0', '#ff4853', '#ffb038', '#7cc668', '#67c2df', '#6773df', '#939fad']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=labels, 
    y=values, 
    marker_color=colores,  # Especifica los colores de las barras
    text=values,  # Texto que se mostrará en las barras 
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    title='Número de unidades por días en el mercado',  # Título de la gráfica
    xaxis=dict(title='Rango días'),  # Título del eje x
    yaxis=dict(
        #title='Cantidad', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_tiempomercado_alfas_tulancingo', carpeta='graficas')
fig.show()

### DIRECTOS

In [13]:
datos_tiempomercado_tulancingo = {
    'dias': ['0-10','11-30','31-100','101-364','>=365'],
    'conteo': [25,50,79,47,30]
}
tiempo_mercado_tulancingo_direct = pd.DataFrame(datos_tiempomercado_tulancingo)
tiempo_mercado_tulancingo_direct

,dias,conteo
0,0-10,25
1,11-30,50
2,31-100,79
3,101-364,47
4,>=365,30


In [14]:
values = tiempo_mercado_tulancingo_direct['conteo']
labels = tiempo_mercado_tulancingo_direct['dias']
colores = ['#3d45c0', '#ff4853', '#ffb038', '#7cc668', '#67c2df', '#6773df', '#939fad']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=labels, 
    y=values, 
    marker_color=colores,  # Especifica los colores de las barras
    text=values,  # Texto que se mostrará en las barras 
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    title='Número de unidades por días en el mercado',  # Título de la gráfica
    xaxis=dict(title='Rango días'),  # Título del eje x
    yaxis=dict(
        title='Unidades', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_tiempomercado_tulancingo', carpeta='graficas')
fig.show()

# *PUEBLA*

In [17]:
# Seleccionar solo los DataFrames que contienen la cadena "pachuca"
dataframes_pachuca = [df for df in dfs_list if 'puebla' in df]
print(dataframes_pachuca)

['df_alfa_q_puebla', 'df_financiamientos_2019_puebla', 'df_financiamientos_2020_puebla', 'df_financiamientos_2021_puebla', 'df_financiamientos_2022_puebla', 'df_financiamientos_2023_puebla', 'df_grupos_edad_puebla', 'df_jul_2023_puebla', 'df_publicacion_puebla', 'df_sep_2023_puebla']


In [20]:
df_mercado_puebla = df_publicacion_puebla[['id','Tiempo_publicacion']]
df_mercado_puebla.head()

,id,Tiempo_publicacion
0,1,1
1,2,8
2,3,5
3,4,16
4,5,16


In [22]:
# Definir la función de categorización
def categorizar(valor):
    valor = int(valor)  # Convertir a entero si es un número
    if 0 <= valor <= 10:
        return '0-10'
    elif 11 <= valor <= 30:
        return '11-30'
    elif 31 <= valor <= 100:
        return '31-100'
    elif 101 <= valor <= 364:
        return '101-364'
    else:
        return '>=365'
# Lista de nombres de los DataFrames
dataframes_name = ['df_mercado_puebla']

# Lista de DataFrames
dataframes_list = [df_mercado_puebla]

# Definir el orden de las categorías
todas_categorias = ['0-10', '11-30', '31-100', '101-364', '>=365']

# Diccionario para almacenar los DataFrames de conteo por nombre de DataFrame original
dataframes_conteo_por_nombre = {}

# Iterar sobre cada DataFrame
for name, df in zip(dataframes_name, dataframes_list):
    # Aplicar la función de categorización
    df['dias'] = df['Tiempo_publicacion'].apply(categorizar)
    # Contar el número de registros únicos por cada valor único en la columna 'dias'
    conteo_por_dias = df['dias'].value_counts().reindex(todas_categorias, fill_value=0)
    # Crear un nuevo DataFrame de conteo
    tiempo_mercado_tulancingo = pd.DataFrame({'dias': conteo_por_dias.index, 'conteo': conteo_por_dias.values})
    # Ordenar el DataFrame según el orden definido en todas_categorias
    tiempo_mercado_tulancingo['dias'] = pd.Categorical(tiempo_mercado_tulancingo['dias'], categories=todas_categorias, ordered=True)
    tiempo_mercado_tulancingo = tiempo_mercado_tulancingo.sort_values(by='dias')
    # Almacenar el DataFrame de conteo en el diccionario por nombre de DataFrame original
    dataframes_conteo_por_nombre[name] = tiempo_mercado_tulancingo
    # Imprimir el nombre del DataFrame de conteo y su contenido
    nuevo_nombre_df_conteo = 'conteo_' + name
    globals()[nuevo_nombre_df_conteo] = tiempo_mercado_tulancingo
    print(f'Nombre del DataFrame de conteo: {nuevo_nombre_df_conteo}')
    print(tiempo_mercado_tulancingo)

Nombre del DataFrame de conteo: conteo_df_mercado_puebla
      dias  conteo
0     0-10    1680
1    11-30     762
2   31-100     703
3  101-364     851
4    >=365     427


C:\Users\yoe11\AppData\Local\Temp\ipykernel_4064\456191811.py:29: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [23]:
values = conteo_df_mercado_puebla['conteo']
labels = conteo_df_mercado_puebla['dias']
colores = ['#3d45c0', '#ff4853', '#ffb038', '#7cc668', '#67c2df', '#6773df', '#939fad']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=labels, 
    y=values, 
    marker_color=colores,  # Especifica los colores de las barras
    text=values,  # Texto que se mostrará en las barras 
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    title='Número de unidades por días en el mercado',  # Título de la gráfica
    xaxis=dict(title='Rango días'),  # Título del eje x
    yaxis=dict(
        #title='Cantidad', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_tiempomercado_alfas_puebla', carpeta='graficas')
fig.show()

### DIRECTOS

In [24]:
datos_tiempomercado_puebla = {
    'dias': ['0-10','11-30','31-100','101-364','>=365'],
    'conteo': [107,613,1391,1420,906]
}
tiempo_mercado_puebla_direct = pd.DataFrame(datos_tiempomercado_puebla)
tiempo_mercado_puebla_direct

,dias,conteo
0,0-10,107
1,11-30,613
2,31-100,1391
3,101-364,1420
4,>=365,906


In [25]:
values = tiempo_mercado_puebla_direct['conteo']
labels = tiempo_mercado_puebla_direct['dias']
colores = ['#3d45c0', '#ff4853', '#ffb038', '#7cc668', '#67c2df', '#6773df', '#939fad']

fig = go.Figure()
fig.add_trace(go.Bar(
    x=labels, 
    y=values, 
    marker_color=colores,  # Especifica los colores de las barras
    text=values,  # Texto que se mostrará en las barras 
    textposition='inside',  # Posición del texto (puede ser 'inside' o 'outside')
))
fig.update_layout(
    title='Número de unidades por días en el mercado',  # Título de la gráfica
    xaxis=dict(title='Rango días'),  # Título del eje x
    yaxis=dict(
        #title='Cantidad', # Título del eje y
        gridcolor='#dddcda', # Color de las líneas que dividen los rangos del eje Y
         gridwidth=1,
    ),  
    plot_bgcolor='rgba(0,0,0,0)',  # Color de fondo del gráfico
)
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_bar_tiempomercado_direct_puebla', carpeta='graficas')
fig.show()

_______
# **RIESGO**

# *PACHUCA*

### BASE

In [7]:
conteo_df_mercado_pachuca

,dias,conteo
0,0-10,162
1,11-30,169
2,31-100,418
3,101-364,905
4,>=365,411


In [8]:
# Definir los valores para la columna 'stock'
stock_values = []
for dias in conteo_df_mercado_pachuca['dias']:
    if dias == '>=365':
        stock_values.append('Más de un año en stock')
    else:
        stock_values.append('Menos de un año en stock')

# Crear el DataFrame 'riesgo'
riesgo = pd.DataFrame({
    'stock': stock_values,
    'casas': conteo_df_mercado_pachuca['conteo']
})

# Agrupar por la columna 'stock' y sumar los valores de 'casas'
riesgo_stock = riesgo.groupby('stock').sum().reset_index()
# Ordenar el DataFrame para que 'Menos de un año en stock' quede abajo
riesgo_stock = riesgo_stock.sort_values(by='stock', ascending=False)
riesgo_stock

,stock,casas
1,Más de un año en stock,411
0,Menos de un año en stock,1654


In [10]:
labels = riesgo_stock['stock']
values = riesgo_stock['casas']
colores = ['#9500ff', '#ff0059']
# Crear el gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, 
    sort=False,    
    marker=dict(colors=colores,)
)])
#fig.update_layout(title="% de riesgo en stock")
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_pie_riesgO_pachuca', carpeta='graficas')
fig.show()

### DIRECTOS

In [29]:
datos = {
    'stock': ['Más de un año en stcok','Menos de un año en stock'],
    'casas': [ 413, 2067 ]
}
# Crear DataFrame
df_riesgo_pachuca = pd.DataFrame(datos)
df_riesgo_pachuca

,stock,casas
0,Más de un año en stcok,413
1,Menos de un año en stock,2067


In [31]:
labels = df_riesgo_pachuca['stock']
values = df_riesgo_pachuca['casas']
colores = ['#3d45c0', '#ff4853']
# Crear el gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, 
    sort=False,    
    marker=dict(colors=colores,)
)])
fig.update_layout(title="% de riesgo en stock")
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_pie_riesgo_direct_pachuca', carpeta='graficas')
fig.show()

# *TULANCINGO*

In [32]:
conteo_df_tiempo_dias_tulancingo

,dias,conteo
0,0-10,0
1,11-30,63
2,31-100,24
3,101-364,0
4,>=365,0


In [33]:
# Definir los valores para la columna 'stock'
stock_values = []
for dias in conteo_df_tiempo_dias_tulancingo['dias']:
    if dias == '>=365':
        stock_values.append('Más de un año en stock')
    else:
        stock_values.append('Menos de un año en stock')

# Crear el DataFrame 'riesgo'
riesgo = pd.DataFrame({
    'stock': stock_values,
    'casas': conteo_df_tiempo_dias_tulancingo['conteo']
})

# Agrupar por la columna 'stock' y sumar los valores de 'casas'
riesgo_stock = riesgo.groupby('stock').sum().reset_index()
# Ordenar el DataFrame para que 'Menos de un año en stock' quede abajo
riesgo_stock = riesgo_stock.sort_values(by='stock', ascending=False)
riesgo_stock

,stock,casas
1,Más de un año en stock,0
0,Menos de un año en stock,87


In [34]:
labels = riesgo_stock['stock']
values = riesgo_stock['casas']
colores = ['#3d45c0', '#ff4853']
# Crear el gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, 
    sort=False,    
    marker=dict(colors=colores,)
)])
fig.update_layout(title="% de riesgo en stock")
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_pie_riesgo_tulancingo', carpeta='graficas')
fig.show()

### DIRECTO

In [35]:
tiempo_mercado_tulancingo_direct

,dias,conteo
0,0-10,25
1,11-30,50
2,31-100,79
3,101-364,47
4,>=365,30


In [36]:
# Definir los valores para la columna 'stock'
stock_values = []
for dias in tiempo_mercado_tulancingo_direct['dias']:
    if dias == '>=365':
        stock_values.append('Más de un año en stock')
    else:
        stock_values.append('Menos de un año en stock')

# Crear el DataFrame 'riesgo'
riesgo = pd.DataFrame({
    'stock': stock_values,
    'casas': tiempo_mercado_tulancingo_direct['conteo']
})

# Agrupar por la columna 'stock' y sumar los valores de 'casas'
riesgo_stock = riesgo.groupby('stock').sum().reset_index()
# Ordenar el DataFrame para que 'Menos de un año en stock' quede abajo
riesgo_stock = riesgo_stock.sort_values(by='stock', ascending=False)
riesgo_stock

,stock,casas
1,Más de un año en stock,30
0,Menos de un año en stock,201


In [37]:
labels = riesgo_stock['stock']
values = riesgo_stock['casas']
colores = ['#3d45c0', '#ff4853']
# Crear el gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, 
    sort=False,    
    marker=dict(colors=colores,)
)])
fig.update_layout(title="% de riesgo en stock")
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_pie_riesgo_direct_tulancingo', carpeta='graficas')
fig.show()

# *PUEBLA*

In [40]:
conteo_df_mercado_puebla

,dias,conteo
0,0-10,1680
1,11-30,762
2,31-100,703
3,101-364,851
4,>=365,427


In [41]:
# Definir los valores para la columna 'stock'
stock_values = []
for dias in conteo_df_mercado_puebla['dias']:
    if dias == '>=365':
        stock_values.append('Más de un año en stock')
    else:
        stock_values.append('Menos de un año en stock')

# Crear el DataFrame 'riesgo'
riesgo = pd.DataFrame({
    'stock': stock_values,
    'casas': conteo_df_mercado_puebla['conteo']
})

# Agrupar por la columna 'stock' y sumar los valores de 'casas'
riesgo_stock = riesgo.groupby('stock').sum().reset_index()
# Ordenar el DataFrame para que 'Menos de un año en stock' quede abajo
riesgo_stock = riesgo_stock.sort_values(by='stock', ascending=False)
riesgo_stock

,stock,casas
1,Más de un año en stock,427
0,Menos de un año en stock,3996


In [42]:
labels = riesgo_stock['stock']
values = riesgo_stock['casas']
colores = ['#3d45c0', '#ff4853']
# Crear el gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, 
    sort=False,    
    marker=dict(colors=colores,)
)])
fig.update_layout(title="% de riesgo en stock")
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_pie_riesgo_puebla', carpeta='graficas')
fig.show()

### DIRECT

In [43]:
tiempo_mercado_puebla_direct

,dias,conteo
0,0-10,107
1,11-30,613
2,31-100,1391
3,101-364,1420
4,>=365,906


In [44]:
# Definir los valores para la columna 'stock'
stock_values = []
for dias in tiempo_mercado_puebla_direct['dias']:
    if dias == '>=365':
        stock_values.append('Más de un año en stock')
    else:
        stock_values.append('Menos de un año en stock')

# Crear el DataFrame 'riesgo'
riesgo = pd.DataFrame({
    'stock': stock_values,
    'casas': tiempo_mercado_puebla_direct['conteo']
})

# Agrupar por la columna 'stock' y sumar los valores de 'casas'
riesgo_stock = riesgo.groupby('stock').sum().reset_index()
# Ordenar el DataFrame para que 'Menos de un año en stock' quede abajo
riesgo_stock = riesgo_stock.sort_values(by='stock', ascending=False)
riesgo_stock

,stock,casas
1,Más de un año en stock,906
0,Menos de un año en stock,3531


In [45]:
labels = riesgo_stock['stock']
values = riesgo_stock['casas']
colores = ['#3d45c0', '#ff4853']
# Crear el gráfico de pastel
fig = go.Figure(data=[go.Pie(
    labels=labels, 
    values=values, 
    sort=False,    
    marker=dict(colors=colores,)
)])
fig.update_layout(title="% de riesgo en stock")
# Exportar gráfica como archivo HTML
def guardar_grafico_como_html(fig, nombre_archivo, carpeta='graficas'):
    # Crear la carpeta si no existe
    if not os.path.exists(carpeta):
        os.makedirs(carpeta)
    
    # Gráfica como archivo HTML en la carpeta especificada
    pio.write_html(fig, f'{carpeta}/{nombre_archivo}.html')

# Exportar
guardar_grafico_como_html(fig, 'g_pie_riesgo_direct_puebla', carpeta='graficas')
fig.show()